# ETL


In [439]:
import pandas as pd
import numpy as np
import ast
import sys

In [440]:
#movies = pd.read_csv("data/movies_dataset.csv")
#credits = pd.read_csv("data/credits.csv")
#movies["popularity"] = movies["popularity"].astype(str)
#movies.to_parquet("parquet_data/movies_parquet.parquet")
#credits.to_parquet("parquet_data/credits_parquet.parquet")

### Reducing down credit dataframe size

In [445]:
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}

In [446]:
credits_parquet = pd.read_parquet("parquet_data/credits_parquet.parquet")

In [447]:
# Dropping heavyweight columns cast
credits_parquet.drop("cast", axis=1, inplace=True)
# Converting all string data into object
credits_parquet["crew"] = credits_parquet["crew"].apply(safe_literal_eval)
# Getting only job position whose value is "Director only"
credits_parquet["crew"] = credits_parquet["crew"].apply(lambda colaborators: [colaborator for colaborator in colaborators if colaborator["job"] == "Director"])
# Getting only name of the director in order to reduce memory usage
credits_parquet["crew"] = credits_parquet["crew"].apply(lambda director: [{"name": attribute["name"]} for attribute in director])
# Renaming the column to a more suitable one
credits_parquet.rename(columns={"crew": "directors"}, inplace=True)
# Converting the id field in string in order to merge this data with the movies data frame
credits_parquet["id"] = credits_parquet["id"].astype(str)

### Movies Data Frame

In [448]:
movies_df = pd.read_parquet("parquet_data/movies_parquet.parquet")

In [449]:
movies_df.shape

(45466, 24)

In [450]:
movies_df.loc[711]

adult                                                                False
belongs_to_collection                                                 None
budget                                                                   0
genres                                                                  []
homepage                                                              None
id                                                                  365371
imdb_id                                                          tt0114894
original_language                                                       en
original_title                        War Stories Our Mother Never Told Us
overview                 Seven New Zealand women speak about their live...
popularity                                                        0.005625
poster_path                                                           None
production_companies                                                    []
production_countries     

In [451]:
movies_df.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,0.0,7.7,5415.0


### Exploring movies data frame.

* Null Values
* Data Types (Its homogeneity)

In [452]:
# Null values
movies_df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   0
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [453]:
# Data Types
movies_df.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                    float64
vote_average             float64
vote_count               float64
dtype: object

### Dropping Fields
Let's begin with the easiest ones: dropping fields! 

In [454]:
movies_df.drop(["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"], axis=1, inplace=True)

In [455]:
movies_df.isnull().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   0
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

### Data Types Consistency
Sometimes, there could be some unexpected data in a field, either a string datatype in a numeric field or a numeric dtype in a string field. It is important to check datatype consistency on each field. And the bellow ad-hoc function will help us to do that a bit easier. 

##### What it does? 

Answer: It returns the indexes of intrusive rows in the series, which it'll be used as a mask. That will allow us to: 
 * Have a visualization of the inconsistencies and 
 * The rows position that we would like to impute them.

In [456]:
def dtype_checker(data: pd.DataFrame, column: str, data_type) -> list:
    """
    Returns an array of indexes of rows with a different data type in the specified column.

    Parameters:
        data (pd.DataFrame or pd.Series): The DataFrame or Series to check.
        column (str): The name of the column to check for data type.
        data_type: The expected data type for the values in the column.

    Returns:
        list: An array of indexes of rows where the data type in the specified column is different from the expected data type.
    """

    invalid_dtype_rows = []

    if isinstance(data, pd.DataFrame):
        for row in data[column].items():
            if not isinstance(row[1], data_type):
                invalid_dtype_rows.append(row[0])
    elif isinstance(data, pd.Series):
        for row in data.items():
            if not isinstance(row[1], data_type):
                invalid_dtype_rows.append(row[0])
    else:
        raise ValueError("Invalid input data type. The data must be a pandas DataFrame or Series.")

    return invalid_dtype_rows

It is usefull to get to know what are the default dtype the dataframe is built of. For this, lets check the data type on a single row in the release_date field

In [457]:
type(movies_df["release_date"][0])

str

Once we get to know the dtype the data is readen, we can see the simple output of the function wich is only  the index position of all that rows with different data type from string

In [458]:
# In the column release_date, the first five instrusive rows position are lited.
dtype_checker(movies_df, column="release_date", data_type=str)[:5] 

[711, 734, 3460, 3628, 5879]

And the above list will be usefull for slicing data and do what may be more comvinience: 

* impute
* drop
* isolate
* etc..

### Creating release_year field in our data frame

1. Check data consistency:
    * Spot intrusive (null, int, floats, etc) data and then drop those rows.
    * Spot any other row that does not match this pattern "1900-01-01"
2. Cast year to create release_year field:
    * With `pd.Series.dt.year()`. It'll only work once we get consistency format



In [459]:
# Spoting intrusive data
mask_for_date_intrusives = dtype_checker(movies_df, column="release_date", data_type=str)
# Dropping rows with nulls
movies_df.drop(mask_for_date_intrusives, inplace=True)

In [460]:
# And, all that null or numeric field are imputed with this default value.
movies_df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0
1,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


In [461]:
movies_df["release_date"].head()

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

But there are also "numbers" in this field, and I named numbers between parentesis because, literally, there could be numbers like "1", "2", "121", etc.. they are recognized from the dtype checker function as string, and we are still going to get issues when casting data as date type. See how we can spot those string numbers by regex expresion.

In [462]:
regex_date = r"^\d{4}-\d{2}-\d{2}$" 
#In this lil line, by adding in the begining this ~ to our array of booleans, we get the opposite from pd.Series.str.contains(any_regex_expression)
movies_df.loc[~movies_df["release_date"].str.contains(regex_date)]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19730,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...",1997-08-20,104.0,Released,nan,False,6.0,1,NaN,NaN,None,None,None,None,NaN,NaN
29503,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",2012-09-29,68.0,Released,nan,False,7.0,12,NaN,NaN,None,None,None,None,NaN,NaN
35587,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",2014-01-01,82.0,Released,Beware Of Frost Bites,False,4.3,22,NaN,NaN,None,None,None,None,NaN,NaN


In [463]:
movies_df.loc[~movies_df["release_date"].str.contains(regex_date)].index

Index([19730, 29503, 35587], dtype='int64')

In [464]:
# So that lets impute these values as well.
movies_df.drop(movies_df.loc[~movies_df["release_date"].str.contains(regex_date)].index, inplace=True)


In [465]:
# Trying if it works.
pd.to_datetime(movies_df["release_date"]).dt.year

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_date, Length: 45376, dtype: int32

# And now, no further issues to cast this column as date type and grab the year only to create our year field.
movies_df['release_year'] = pd.to_datetime(movies_df["release_date"]).dt.year

In [466]:
# lets take a look at the data frame movies
movies_df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0
1,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


### Filling null values on "revenue" and "budget"

According to henry's guidance project, We should fill numeric missing data with zeros.

Steps:

    1. Check consistency:
        * Data type
        * Format (Regex)
        * Nulls
    2. Impute and convert

In [467]:
# Revenue field data type
movies_df["revenue"].dtype

dtype('float64')

In [468]:
movies_df["revenue"].isnull().sum()

0

In [469]:
# Check how many rows aren't: float, int, str. Recall that movies_df has 45466 rows so far.
len(dtype_checker(movies_df, column="revenue", data_type=float)), len(dtype_checker(movies_df, column="revenue", data_type=int)), len(dtype_checker(movies_df, column="revenue", data_type=str))

(0, 45376, 45376)

Ok so.. We have 0 values that aren't floats, 45466 aren't integers, 45466 aren't str, thus, all values are float. Good

In [470]:
movies_df["revenue"] / 2

0        186777016.5
1        131398624.5
2                0.0
3         40726078.0
4         38289455.5
            ...     
45460            0.0
45462            0.0
45463            0.0
45464            0.0
45465            0.0
Name: revenue, Length: 45376, dtype: float64

In [471]:
# Budget field data type
movies_df["budget"].dtype

dtype('O')

In [472]:
movies_df['budget'].isnull().sum()

0

In [473]:
len(dtype_checker(movies_df, column="budget", data_type=float)), len(dtype_checker(movies_df, column="budget", data_type=int)), len(dtype_checker(movies_df, column="budget", data_type=str))

(45376, 45376, 0)

Ok so.. We have 45466 values that aren't floats, 45466 aren't integers, 0 aren't str, thus, all values are string. Not too good. Let's just try to cast this as float and see what will happen 

In [474]:
movies_df["budget"].astype(float)

0        30000000.0
1        65000000.0
2               0.0
3        16000000.0
4               0.0
            ...    
45460           0.0
45462           0.0
45463           0.0
45464           0.0
45465           0.0
Name: budget, Length: 45376, dtype: float64

In [475]:
movies_df["budget"] = movies_df["budget"].astype(float)

In [476]:
movies_df["budget"].dtype, movies_df["budget"].isnull().sum()

(dtype('float64'), 0)

### Creating "return_on_investment" (ROI) fiel

This is what is performed.. divide revenue by budget as float, then fill null values with zero.. after filling nulls replace inifinites by zeros.

In [477]:
movies_df["revenue"].div(movies_df["budget"].astype(float)).fillna(0).replace([np.inf, -np.inf], 0)

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Length: 45376, dtype: float64

In [478]:
movies_df["return_on_investment"] = movies_df["revenue"].div(movies_df["budget"].astype(float)).fillna(0).replace([np.inf, -np.inf], 0)

In [479]:
movies_df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801
1,None,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000
3,None,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000


### Flatting Nested Data

Fields such as belongs_to_collection and genres have relevant data, needed for our API, EDA, and ML model. So lets wrangling and format those fields in order to allow us to query and manipulate as needed.

Lets begin with belongs_to_collection field. It seems that this is a field with few missing values and this is because not all movies belongs to a collection such as James Bon's movies. Also, each row contains a dictionary.

#### The approach

Data Transformation:

        * Explore data in field (dtype)
        * Create a new data frame for bellongs_to_collection
        * Convert data into the original object (dictionary)


### Data Modeling on belongs_to_collection Field

In [480]:
# I always get a views of any random objectn in the field in order to have an idea of what is in it.
movies_df["belongs_to_collection"][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [481]:
# What type of object does the field got?
movies_df.loc[:,"belongs_to_collection"].dtype

dtype('O')

In [482]:
movies_df["belongs_to_collection"].isnull().sum()

40888

In [483]:
# Building the new data frame for belongs_to_collection
belongs_to_collections_df = movies_df.loc[movies_df["belongs_to_collection"].isnull() == False, "belongs_to_collection"]
belongs_to_collections_df.shape

(4488,)

Since belongs_to_collection only contains "object" types, we need to convert those pandas' object types into dictionary by `ast.literal_eval()` wich grabs the string and catch it as the data format that seems to be. 

For example: "{'hello': 2}" string --> {'hello': 2} dict. And it will enable all the dictionary methods needed for this data manipulation

In [484]:
# This functionn is only for handling any expected error and impute with default empty list
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}

In [485]:
belongs_to_collections_df = belongs_to_collections_df.apply(safe_literal_eval)

In [486]:
belongs_to_collections_df.head()[0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

In [487]:
belongs_to_collections_df

0        {'id': 10194, 'name': 'Toy Story Collection', ...
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
4        {'id': 96871, 'name': 'Father of the Bride Col...
9        {'id': 645, 'name': 'James Bond Collection', '...
12       {'id': 117693, 'name': 'Balto Collection', 'po...
                               ...                        
45355    {'id': 37261, 'name': 'The Carry On Collection...
45358    {'id': 37261, 'name': 'The Carry On Collection...
45369    {'id': 37261, 'name': 'The Carry On Collection...
45371    {'id': 477208, 'name': 'DC Super Hero Girls Co...
45382    {'id': 200641, 'name': 'Red Lotus Collection',...
Name: belongs_to_collection, Length: 4488, dtype: object

In [488]:
collections_df = pd.DataFrame(belongs_to_collections_df.tolist())

In [489]:
collections_df.drop_duplicates(subset=["id"], keep='first', inplace=True)

In [490]:
collections_df.drop(["poster_path", "backdrop_path"], axis=1, inplace=True)

In [491]:
collections_df

,id,name
0,10194,Toy Story Collection
1,119050,Grumpy Old Men Collection
2,96871,Father of the Bride Collection
3,645,James Bond Collection
4,117693,Balto Collection
...,...,...
4464,104774,Tomtar och Trolltyg Collection
4466,400500,Чебурашка и крокодил Гена
4472,148603,Ducobu Collection
4479,152918,Mister Blot Collection


In [492]:
collections_df.rename(columns={"id": "collection_id"}, inplace=True)

In [493]:
collections_df["collection_id"] = collections_df["collection_id"].astype(str)

### Getting Id's from belongs_to_collection field

In [494]:
# In order to deal with null values, impute those records with an default object is conveniant when is time to process each object without further dtype issues.
# Is like adding fake consistency, letter, we can spot those missing data to its origin dtype
default_collection = "{'id': 1234567, 'name': 'Null', 'poster_path': 'im.jpg', 'backdrop_path': 'im.jpg'}"
movies_df.loc[movies_df["belongs_to_collection"].isna(), "belongs_to_collection"] = default_collection
# Getting only the Ids
movies_df["belongs_to_collection"] = movies_df["belongs_to_collection"].apply(ast.literal_eval).apply(lambda x: x["id"]).astype(str)
# Imputing to nulls
movies_df.loc[movies_df["belongs_to_collection"] == "1234567", "belongs_to_collection"] = np.nan
# Renaming belongs_to_collection field to collection_id to get consistency in field name when mergin datasets
movies_df.rename(columns={"belongs_to_collection": "collection_id"}, inplace=True)

### Trying out our collectiond_df and new collection_id field in movies_df

In [495]:
collections_df.merge(movies_df[["collection_id", "title", "revenue"]], on="collection_id")

,collection_id,name,title,revenue
0,10194,Toy Story Collection,Toy Story,3.735540e+08
1,10194,Toy Story Collection,Toy Story 2,4.973669e+08
2,10194,Toy Story Collection,Toy Story 3,1.066970e+09
3,119050,Grumpy Old Men Collection,Grumpier Old Men,0.000000e+00
4,119050,Grumpy Old Men Collection,Grumpy Old Men,0.000000e+00
...,...,...,...,...
4483,400500,Чебурашка и крокодил Гена,Gena the Crocodile,0.000000e+00
4484,148603,Ducobu Collection,Ducoboo,0.000000e+00
4485,152918,Mister Blot Collection,Mister Blot's Academy,0.000000e+00
4486,152918,Mister Blot Collection,Mr. Blot in Space,0.000000e+00


### Flattent genres field

This is going to be a bit more complex since in this case we have a many objects related to one row, and many rows releted to one object.. In other words, a reletion ship many to many. 

#### The Approach:

Normaliza genres:

        * Build a new data frame for genres:
                * Cast any single row in genres as lists
                * Check data consistency and impute
        * Extract information:
                * Explode genres field.
                * Drop duplicates
        * Drop duplicates

Data Transformation for genres field in movies_df:

        * Grab only ids

In [496]:
movies_df.head(2)

,collection_id,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
0,10194,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035


In [497]:
movies_df["genres"][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [498]:
empty_list_pattern = r'^\[\]$'
genre_empy_mask = movies_df["genres"].astype(str).str.match(empty_list_pattern)
print(genre_empy_mask.sum())
movies_df[genre_empy_mask].head(2)

2384


,collection_id,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
55,NaN,0.0,[],124057,en,"Set in modern times, Alex finds King Arthur's ...",0.307075,"[{'name': 'Telefilm Canada', 'id': 7320}, {'na...",[],1997-07-08,0.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Kids of the Round Table,3.0,1.0,0.0
83,NaN,0.0,[],188588,en,"Filmed entirely on location in East Hampton, L...",0.531159,[],[],1995-11-22,0.0,108.0,[],Released,None,Last Summer in the Hamptons,0.0,0.0,0.0


In [499]:
movies_df.loc[genre_empy_mask, "genres"] = "[{'id': 123456, 'name': 'Unknown'}]"

In [500]:
movies_df.loc[genre_empy_mask, "genres"]

55       [{'id': 123456, 'name': 'Unknown'}]
83       [{'id': 123456, 'name': 'Unknown'}]
126      [{'id': 123456, 'name': 'Unknown'}]
137      [{'id': 123456, 'name': 'Unknown'}]
390      [{'id': 123456, 'name': 'Unknown'}]
                        ...                 
45447    [{'id': 123456, 'name': 'Unknown'}]
45448    [{'id': 123456, 'name': 'Unknown'}]
45455    [{'id': 123456, 'name': 'Unknown'}]
45464    [{'id': 123456, 'name': 'Unknown'}]
45465    [{'id': 123456, 'name': 'Unknown'}]
Name: genres, Length: 2384, dtype: object

In [501]:
movies_df["genres"] = movies_df["genres"].apply(safe_literal_eval)

In [502]:
movies_df["genres"][0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [503]:
movies_df.explode('genres', ignore_index=True).head(2)#[["genres"]].rename(columns={"genres": "genres_info"})

,collection_id,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
0,10194,30000000.0,"{'id': 16, 'name': 'Animation'}",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801
1,10194,30000000.0,"{'id': 35, 'name': 'Comedy'}",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801


In [504]:
movies_df.explode('genres', ignore_index=True).tail(2)#[["genres"]].rename(columns={"genres": "genres_info"})

,collection_id,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
93418,NaN,0.0,"{'id': 123456, 'name': 'Unknown'}",227506,en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],Released,None,Satan Triumphant,0.0,0.0,0.0
93419,NaN,0.0,"{'id': 123456, 'name': 'Unknown'}",461257,en,50 years after decriminalisation of homosexual...,0.163015,[],"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Queerama,0.0,0.0,0.0


In [505]:
genres_df = movies_df.explode('genres', ignore_index=True)[["genres"]].rename(columns={"genres": "genres_info"})

In [506]:
genres_df

,genres_info
0,"{'id': 16, 'name': 'Animation'}"
1,"{'id': 35, 'name': 'Comedy'}"
2,"{'id': 10751, 'name': 'Family'}"
3,"{'id': 12, 'name': 'Adventure'}"
4,"{'id': 14, 'name': 'Fantasy'}"
...,...
93415,"{'id': 28, 'name': 'Action'}"
93416,"{'id': 18, 'name': 'Drama'}"
93417,"{'id': 53, 'name': 'Thriller'}"
93418,"{'id': 123456, 'name': 'Unknown'}"


In [507]:
genres_df["genre_id"] = genres_df["genres_info"].apply(lambda genre: genre["id"])
genres_df["genre_id"] = genres_df["genre_id"].astype(str)

genres_df["genre_name"] = genres_df["genres_info"].apply(lambda genre: genre["name"])
genres_df.drop(columns="genres_info", inplace=True)
genres_df

,genre_id,genre_name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
...,...,...
93415,28,Action
93416,18,Drama
93417,53,Thriller
93418,123456,Unknown


In [508]:
genres_df.drop_duplicates(subset=["genre_id"], keep="first", inplace=True)

In [509]:
movies_genres_df = movies_df.explode('genres', ignore_index=True)[['id', 'genres']].rename(columns={'genres': 'genre_info'})
movies_genres_df["genre_id"] = movies_genres_df["genre_info"].apply(lambda genre: genre["id"])
movies_genres_df["genre_id"] = movies_genres_df["genre_id"].astype(str)

In [510]:
movies_genres_df.drop(columns="genre_info", inplace=True)

In [511]:
movies_genres_df.merge(movies_df[["id", "title"]], on='id').merge(genres_df, on="genre_id")

,id,genre_id,title,genre_name
0,862,16,Toy Story,Animation
1,21032,16,Balto,Animation
2,10530,16,Pocahontas,Animation
3,15789,16,A Goofy Movie,Animation
4,43475,16,Gumby: The Movie,Animation
...,...,...,...,...
93577,69560,10770,Dying to Belong,TV Movie
93578,417320,10770,Descendants 2,TV Movie
93579,458618,10770,Ivanka Trump- America's Real First Lady?,TV Movie
93580,401687,10770,Hopeless Romantic,TV Movie


In [512]:
movies_df.drop(labels=["genres"], axis=1, inplace=True)

In [513]:
movies_df.columns

Index(['collection_id', 'budget', 'id', 'original_language', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count',
       'return_on_investment'],
      dtype='object')

### Production Companies Field

In [514]:
movies_df["production_companies"][3]

"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]"

In [515]:
empty_list_pattern = r'^\[\]$'
print(movies_df["production_companies"].str.match(empty_list_pattern).sum())
production_movies_mask = movies_df["production_companies"].str.match(empty_list_pattern)
production_movies_mask

11796


0        False
1        False
2        False
3        False
4        False
         ...  
45460    False
45462    False
45463    False
45464    False
45465     True
Name: production_companies, Length: 45376, dtype: bool

In [516]:
movies_df[production_movies_mask].head(2)

,collection_id,budget,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
50,NaN,0.0,117164,en,Detective - turned - bodyguard Cynthia McKay (...,0.595949,[],[],1994-11-22,0.0,93.0,[],Released,She's no angel of mercy.,Guardian Angel,6.3,3.0,0.0
52,NaN,0.0,49133,it,"Fiore, an Italian conman, arrives in post Comm...",1.361286,[],"[{'iso_3166_1': 'IT', 'name': 'Italy'}, {'iso_...",1994-01-01,0.0,116.0,"[{'iso_639_1': 'sq', 'name': 'shqip'}, {'iso_6...",Released,None,Lamerica,7.7,11.0,0.0


In [517]:
movies_df.loc[production_movies_mask, "production_companies"] = "[{'name': 'Unknown', 'id': 123456}]"

In [518]:
movies_df.loc[production_movies_mask, "production_companies"]

50       [{'name': 'Unknown', 'id': 123456}]
52       [{'name': 'Unknown', 'id': 123456}]
57       [{'name': 'Unknown', 'id': 123456}]
58       [{'name': 'Unknown', 'id': 123456}]
83       [{'name': 'Unknown', 'id': 123456}]
                        ...                 
45442    [{'name': 'Unknown', 'id': 123456}]
45452    [{'name': 'Unknown', 'id': 123456}]
45455    [{'name': 'Unknown', 'id': 123456}]
45457    [{'name': 'Unknown', 'id': 123456}]
45465    [{'name': 'Unknown', 'id': 123456}]
Name: production_companies, Length: 11796, dtype: object

In [519]:
movies_df["production_companies"] = movies_df["production_companies"].apply(safe_literal_eval)

In [520]:
production_companies_df = movies_df.explode("production_companies", ignore_index=True)[["production_companies"]].rename(columns={"production_companies": "production_companies_info"})

In [521]:
production_companies_df["company_name"] = production_companies_df["production_companies_info"].apply(lambda prod_company: prod_company["name"])
production_companies_df["company_id"] = production_companies_df["production_companies_info"].apply(lambda prod_company: prod_company["id"])
production_companies_df.drop("production_companies_info", axis=1, inplace=True)

In [522]:
production_companies_df.drop_duplicates(subset=["company_id"], keep="first", inplace=True)

In [523]:
production_companies_df.dtypes

company_name    object
company_id       int64
dtype: object

In [524]:
production_companies_df["company_id"] = production_companies_df["company_id"].astype(str)

In [525]:
production_companies_df.dtypes

company_name    object
company_id      object
dtype: object

DataFrame for the many-to-many relationship

In [526]:
movies_production_companies_df = movies_df.explode("production_companies", ignore_index=True)[["id", "production_companies"]].rename(columns={"production_companies": "production_companies_info"})
movies_production_companies_df["company_id"] = movies_production_companies_df["production_companies_info"].apply(lambda prod_company: prod_company["id"])
movies_production_companies_df.drop("production_companies_info", axis=1, inplace=True)
movies_production_companies_df["company_id"] = movies_production_companies_df["company_id"].astype(str)
movies_production_companies_df.head()

,id,company_id
0,862,3
1,8844,559
2,8844,2550
3,8844,10201
4,15602,6194


In [527]:
movies_production_companies_df.head(2)

,id,company_id
0,862,3
1,8844,559


### Trying it out

In [530]:
company_demo = movies_production_companies_df.merge(movies_df[["id", "title"]], on = "id").merge(production_companies_df, on="company_id")

In [531]:
company_demo.loc[(company_demo["title"].str.contains("")) & (company_demo["company_name"] == "Pixar Animation Studios")]

,id,company_id,title,company_name
0,862,3,Toy Story,Pixar Animation Studios
1,9487,3,A Bug's Life,Pixar Animation Studios
2,863,3,Toy Story 2,Pixar Animation Studios
3,585,3,"Monsters, Inc.",Pixar Animation Studios
4,12,3,Finding Nemo,Pixar Animation Studios
5,9806,3,The Incredibles,Pixar Animation Studios
6,13925,3,Luxo Jr.,Pixar Animation Studios
7,920,3,Cars,Pixar Animation Studios
8,2062,3,Ratatouille,Pixar Animation Studios
9,10681,3,WALL·E,Pixar Animation Studios


In [532]:
movies_df.drop("production_companies", axis=1, inplace=True)


### Production Companies Field

For only learning purpuses, the approach to tackle this down will be completely different from normalizing data and building tables. Here the approach is going to be the accesing approach by `.apply()` method and lambda expressions against nested data. This leverage the processecing resources rather than the store recourses because we aren't going to create any additionally dataframe.

How it works?

* First of all, we should take care about missing value, in this case they are represented as empy list and levearing this data is currently in string datatype, it will be easier to spot those empty list with regex patterns and then impute a default value to simply label them as "unknowns"
* Once we get all rows in production_companies field with an a list with at least one dictionary object, we should be able to apply our safe_literal_eval to convert those strings into the appropiate objects
* Then, we could filter information over these nested data (dictionaries in a list) with `.apply()` method wich applies any function on each record. We can use eaither custom funcions (for example, safe_literal_eval is one custom function applied on each record in the production_companies field) or lamba expressions.

In [533]:
movies_df.head(2)

,collection_id,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
0,10194,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801
1,NaN,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035


In [534]:
movies_df["production_countries"].head(10)

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
5    [{'iso_3166_1': 'US', 'name': 'United States o...
6    [{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...
7    [{'iso_3166_1': 'US', 'name': 'United States o...
8    [{'iso_3166_1': 'US', 'name': 'United States o...
9    [{'iso_3166_1': 'GB', 'name': 'United Kingdom'...
Name: production_countries, dtype: object

In [535]:
movies_df["production_countries"].str.match(empty_list_pattern).sum()
production_countries_empties_mask = movies_df["production_countries"].str.match(empty_list_pattern)

In [536]:
movies_df.loc[production_countries_empties_mask, "production_countries"] = "[{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]"

In [537]:
movies_df.loc[production_countries_empties_mask, "production_countries"]

50       [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
55       [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
83       [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
106      [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
107      [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
                              ...                      
45432    [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
45434    [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
45438    [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
45455    [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
45457    [{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]
Name: production_countries, Length: 6211, dtype: object

In [538]:
movies_df["production_countries"] = movies_df["production_countries"].apply(safe_literal_eval)

In [539]:
movies_df.head(1)

,collection_id,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
0,10194,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801


In [540]:
movies_df.loc[(movies_df["production_countries"].apply(lambda country: "Mexico" in {item["name"] for item in country}))
              & (movies_df["original_language"] == "en")
              & (movies_df["production_countries"].apply(lambda country_len: len(country_len) > 1))
                , ["original_language", "title", "production_countries"]]

,original_language,title,production_countries
678,en,Solo,"[{'iso_3166_1': 'US', 'name': 'United States o..."
730,en,The Arrival,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}, {'iso..."
1279,en,Amityville II: The Possession,"[{'iso_3166_1': 'IT', 'name': 'Italy'}, {'iso_..."
2124,en,One Man's Hero,"[{'iso_3166_1': 'US', 'name': 'United States o..."
2293,en,Romancing the Stone,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}, {'iso..."
...,...,...,...
40972,en,Western,"[{'iso_3166_1': 'US', 'name': 'United States o..."
42135,en,You're Killing Me Susana,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
42460,en,Flight of the Butterflies,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'..."
44393,en,Jeremiah Tower: The Last Magnificent,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}, {'iso..."


In [541]:
movies_df.loc[(movies_df["production_countries"].apply(lambda country: "Mexico" in {item["name"] for item in country}))
              
              & (movies_df["production_countries"].apply(lambda country_len: len(country_len) == 1))
                , ["id", "original_language", "title", "production_countries"]]

,id,original_language,title,production_countries
261,18183,es,Like Water for Chocolate,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
559,11655,en,Cronos,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
1697,18205,es,Midaq Alley,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
4108,55,es,Amores perros,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
4529,19236,en,Santa Sangre,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
...,...,...,...,...
44526,264389,en,Tropical Carmine,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
44627,387805,es,7:19,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
44734,366860,es,El Violetero,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"
45188,346225,es,Bleak Street,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]"


# Spoken Language field

In [564]:
movies_df["spoken_languages"][1]

[{'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'fr', 'name': 'Français'}]

In [565]:
spoken_languages_mask = movies_df["spoken_languages"].str.match(empty_list_pattern)

In [ ]:
movies_df.loc[spoken_languages_mask, "spoken_languages"] = "[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]"
movies_df["spoken_languages"] = movies_df["spoken_languages"].apply(safe_literal_eval)

# Building Functions

In [551]:
movies_df.head(1)

,collection_id,budget,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return_on_investment
0,10194,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,12.451801


In [552]:
def count_movies_by_original_languages(language: str):

    return {"number of movie": movies_df.loc[movies_df["original_language"] == language].shape[0]}

In [553]:
def get_runtime_and_release_year(movie_title: str):

    runtime_movie = movies_df.loc[movies_df["title"] == movie_title, ["runtime", "release_year"]].values[0][0]
    release_year_movie = movies_df.loc[movies_df["title"] == movie_title, ["runtime", "release_year"]].values[0][1]
    return {"Duracion": runtime_movie, "Año": release_year_movie}

In [554]:
def get_collection_information_by_title(title1):

    # Getting the collection id
    id_of_collection = movies_df.loc[movies_df["title"] == title1, "collection_id"].tolist()[0]

    # Usage of the collection id in the collections and movies merge
    movies_in_collection = collections_df.merge(movies_df[["collection_id", "title", "revenue"]], on="collection_id")

    # Getting name, number of movies, total revenue, and mean revenue for output
    collection_name = movies_in_collection[movies_in_collection["collection_id"] == id_of_collection]["name"].unique()[0]
    number_of_movies = movies_in_collection[movies_in_collection["collection_id"] == id_of_collection].shape[0]
    total_revenue = movies_in_collection[movies_in_collection["collection_id"] == id_of_collection]["revenue"].sum()
    mean_revenue = np.mean(movies_in_collection[movies_in_collection["collection_id"] == id_of_collection]["revenue"])

    return {"Collection name": collection_name, "number_of_movies": number_of_movies, "total_revenue": total_revenue, "mean_revenue": mean_revenue}

In [556]:
def number_of_movies_produced_in_country(country_name):
    
    number_of_movies =  movies_df[movies_df["production_countries"].apply(lambda country: country_name in {item["name"] for item in country})].shape[0]

    return {"country_name": country_name, "numbers_of_movies_produced": number_of_movies}

In [558]:
def production_company_success(production_company_name):

    production_companies = movies_production_companies_df.merge(movies_df[["id","revenue"]], on = "id").merge(production_companies_df[["company_id"]], on="company_id")
    revenue = production_companies.loc[(company_demo["company_name"] == production_company_name), "revenue"].sum()
    number_of_movies = production_companies.loc[(company_demo["company_name"] == production_company_name)].shape[0]

    return {"production_company_name": production_company_name, "total_revenue": revenue, "number_of_movies_produced": number_of_movies}

In [561]:
def director_success(director_name):
    
    mask_director = (credits_parquet.merge(movies_df[["id"]], on="id")
                    .loc[:, "directors"].apply(lambda directors: director_name in {director["name"] for director in directors}))
    
    avg_roi = np.mean(credits_parquet.merge(movies_df[["id","return_on_investment"]], on="id").loc[mask_director, "return_on_investment"])
    
    movies = (credits_parquet.merge(movies_df[["id", "title", "release_date", "return_on_investment", "budget", "revenue"]], on="id")
              .loc[mask_director, ["title", "release_date", "return_on_investment", "budget", "revenue"]].to_dict(orient="records"))

    
    return {"director_name": director_name, "avg_roi": avg_roi, "movies": movies}

In [130]:
import sys

# Get a list of all variable names in the current scope
all_vars = locals()

# Calculate the memory usage of each variable and store the results in a dictionary
memory_usage = {}
for var_name, var_value in all_vars.items():
    memory_usage[var_name] = sys.getsizeof(var_value)

# Print the memory usage of each variable
for var_name, mem_usage in memory_usage.items():
    print(f"{var_name}: {mem_usage} bytes")

__name__: 57 bytes
__doc__: 113 bytes
__package__: 16 bytes
__loader__: 16 bytes
__spec__: 16 bytes
__builtin__: 72 bytes
__builtins__: 72 bytes
_ih: 1240 bytes
_oh: 2272 bytes
_dh: 120 bytes
In: 1240 bytes
Out: 2272 bytes
get_ipython: 64 bytes
exit: 48 bytes
quit: 48 bytes
open: 144 bytes
_: 11256 bytes
__: 28 bytes
___: 232 bytes
__vsc_ipynb_file__: 104 bytes
_i: 134 bytes
_ii: 643 bytes
_iii: 111 bytes
_i1: 109 bytes
pd: 72 bytes
np: 72 bytes
ast: 72 bytes
sys: 72 bytes
_i2: 315 bytes
_i3: 174 bytes
safe_literal_eval: 144 bytes
_i4: 122 bytes
credits_parquet: 3981072 bytes
_i5: 89 bytes
_5: 24 bytes
_i6: 99 bytes
_i7: 89 bytes
_7: 24 bytes
_i8: 123 bytes
_i9: 89 bytes
_9: 24 bytes
_i10: 208 bytes
_i11: 89 bytes
_11: 24 bytes
_i12: 75 bytes
_12: 88 bytes
_i13: 178 bytes
_i14: 89 bytes
_14: 24 bytes
_i15: 64 bytes
_15: 3981072 bytes
_i16: 102 bytes
_16: 3981072 bytes
_i17: 116 bytes
_i18: 53 bytes
_i19: 474 bytes
all_vars: 9312 bytes
memory_usage: 2272 bytes
var_name: 61 bytes
var_val

In [131]:
import sys

# Get a list of all variable names in the current scope
all_vars = locals()

# Calculate the memory usage of each variable and store the results in a dictionary
total_memory_usage = 0
for var_name, var_value in all_vars.items():
    total_memory_usage += sys.getsizeof(var_value)

# Print the memory usage of each variable
print(total_memory_usage / 1000000)

240.415158


In [566]:
!jupyter nbconvert --to script etl.ipynb

[NbConvertApp] Converting notebook etl.ipynb to script
[NbConvertApp] Writing 24895 bytes to etl.py
